In [ ]:
%load_ext autoreload
%autoreload 2
import sys
# sys.path.insert(0, '..')
from Positioner2D import *
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.constants import c
from tqdm.notebook import tqdm

pd.options.plotting.backend = "plotly"

In [ ]:

jsonfile = "C:/Users/gak0n/OneDrive/Desktop/TDOA/uc2-mr3a_20240220_1540_PINPOINT.json"
P= Positioner(jsonfile=jsonfile)


## Positiong Algorithms

In [30]:
P.process_dataframe_positioning(algoType="algorithmLSE",
                                timeWindow=0.5,
                                timeBased=False,
                                )
P.plot_measured_tdoa_map([P.PositionDf['x'], P.PositionDf['y']])

Calculating positions nelder-mead: 100%|██████████| 206/206 [00:11<00:00, 17.49it/s] 


## DOP Simulation for antenna signal radiation 

In [29]:
P.plot_dop_contours()

In [28]:
def calculate_weight(relative_angle, growth_rate):
    if 0 <= relative_angle < 90 or 270 <= relative_angle <= 360:
        return 1  # Good signal weight from 0 to 90
    elif 90 <= relative_angle < 180:
        return 1 - 1 / (1 + np.exp(-growth_rate * (relative_angle - 135)))
    elif 180 < relative_angle <= 270:
        return 1 / (1 + np.exp(-growth_rate * (relative_angle - 225)))

# Simulation parameters
growth_rate = 0.4
angles = np.arange(361)
weights = [calculate_weight(angle, growth_rate) for angle in angles]

# Plot the simulation using Plotly
fig = go.Figure(data=go.Scatter(x=angles, y=weights, mode='lines'))
fig.update_layout(
    title='Weight Simulation',
    xaxis_title='Relative Angle',
    yaxis_title='Weight',
    xaxis=dict(dtick=45),  # Adjust x-axis ticks as needed
    yaxis=dict(range=[0, 1])  # Set y-axis range from 0 to 1
)
fig.show()


## X, Y, Z position over time

In [27]:
P.PositionDf.plot(x="time", y=['x','y','z'])

In [26]:
P.TagPositions.plot(x="index", y=['position.x','position.y','position.z'])

## TDOA computed from reference positions of anchors

In [25]:
P.theoretical_tdoa_map()

## Exploration territory - Position calculations 

In [ ]:
import plotly.express as px


def plotGDOP(addresses, coordinates =None, title ="", xRange=[-20,20], yRange = [-20,20], dopType ="gdop"):

    if coordinates is None:
        coordinates = []
        for addr in addresses:
            coordinates.append(posCoord2list(addr, P.posDict))


    xRange = np.linspace(xRange[0],xRange[1],101)
    yRange = np.linspace(yRange[0],yRange[1],101)
    z = 1
    heatmap = []

    for y in yRange:
        heatmapTemp = []
        for x in xRange:
            results = gdop_calculation([x,y,z], coordinates)
            heatmapTemp.append(results[dopType])

        heatmap.append(heatmapTemp)

    heatmap = np.array(heatmap)

    fig = px.imshow(heatmap,  zmin=0, zmax=50,
                    x = xRange, y= yRange)

    for addr, pos in P.posDict.items():
        addr = int(addr)
        if addr in addresses:
            fig.add_trace(
                go.Scatter(
                        x=[pos['PosX']],  y = [pos['PosY']],
                        mode="markers",
                        name = f"Address: 0x{addr:02X}", ))
    fig.update_layout(legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ))
    fig.update_layout(title=title)
    fig.show()

In [ ]:
addresses = [0x07, 0X9,0xA,0XB, 0X0C]
plotGDOP(addresses, title ="GDOP", dopType="gdop")

In [ ]:
addresses = [0x07, 0X0B,0X09,0x0A]
# https://en.wikipedia.org/wiki/Dilution_of_precision_(navigation)
plotGDOP(addresses, title ="gdop", dopType="gdop")
plotGDOP(addresses, title ="hdop", dopType="hdop")
plotGDOP(addresses, title ="vdop", dopType="vdop")
plotGDOP(addresses, title ="pdop", dopType="pdop")
plotGDOP(addresses, title ="tdop", dopType="tdop")

### Bancroft implementation
No working as the this implementation needs toa and not tdoa

In [ ]:
from dataclasses import dataclass
import numpy as np
from random import randrange
import math

M = np.diag([1, 1, 1, -1])

@dataclass
class Vertexer:

    nodes: np.ndarray

    # Defaults
    v = 299792

    def __post_init__(self):
        # Calculate valid input range
        max = 0
        min = 1E+10
        centroid = np.average(self.nodes, axis = 0)
        for n in self.nodes:
            dist = np.linalg.norm(n - centroid)
            if dist < min:
                min = dist

            for p in self.nodes:
                dist = np.linalg.norm(n - p)

                if dist > max:
                    max = dist

        max /= self.v
        min /= self.v

        print(min, max)

    def errFunc(self, point, times):
        # Return RSS error
        error = 0

        for n, t in zip(self.nodes, times):
            error += ((np.linalg.norm(n - point) / self.v) - t)**2

        return error

    def find(self, times):
        def lorentzInner(v, w):
            # Return Lorentzian Inner-Product
            return np.sum(v * (w @ M), axis = -1)

        A = np.append(self.nodes, times * self.v, axis = 1)

        b = 0.5 * lorentzInner(A, A)
        oneA = np.linalg.solve(A, np.ones(4))
        invA = np.linalg.solve(A, b)

        solution = []
        for Lambda in np.roots([ lorentzInner(oneA, oneA),
                                (lorentzInner(oneA, invA) - 1) * 2,
                                 lorentzInner(invA, invA),
                                ]):
            X, Y, Z, T = M @ np.linalg.solve(A, Lambda * np.ones(4) + b)
            solution.append(np.array([X,Y,Z]))

        return min(solution, key = lambda err: self.errFunc(err, times))

### Bancroft usage

In [ ]:
x_1, y_1, z_1 = P.get_position(P.addresses[0])
x_2, y_2, z_2 = P.get_position(P.addresses[1])
x_3, y_3, z_3 = P.get_position(P.addresses[2])
x_4, y_4, z_4 = P.get_position(P.addresses[3])


t_1 = P.pairsDfs[P.addresses[0],P.addresses[1]]['tdoa'][50]
t_2 = P.pairsDfs[P.addresses[0],P.addresses[2]]['tdoa'][50]
t_3 = P.pairsDfs[P.addresses[0],P.addresses[3]]['tdoa'][50]
t_4 = P.pairsDfs[P.addresses[1],P.addresses[2]]['tdoa'][50]


print(t_1,t_2,t_3, t_4)
# Set velocity
# c = 299792 # km/ns

myVertexer = Vertexer(np.array([[x_1, y_1, z_1],[x_2, y_2, z_2],[x_3, y_3, z_3],[x_4, y_4, z_4]]))
bancroft_pos = myVertexer.find(np.array([[t_1], [t_2], [t_3], [t_4], ]))

print(bancroft_pos)



## Calculations for TDOA positioning

* Matlab calculation of TDOA: https://de.mathworks.com/help/comm/ug/uwb-localization-using-ieee-802.15.4z.html
* Example of solving GPS equations: https://mason.gmu.edu/~treid5/Math447/GPSEquations/
* Interesting Thesis about UWB TDOA solving:https://publikationen.bibliothek.kit.edu/1000036616/3753726
* Bancrof Method calculation:
  *  https://gssc.esa.int/navipedia/index.php?title=Bancroft_Method
  * https://www.yumpu.com/en/document/read/29687499/solving-the-navigation-equation-using-bancrofts-method
* Bancrofts implementation in python: https://codereview.stackexchange.com/questions/253982/bancrofts-method-implementation
* 
* https://lo.calho.st/posts/tdoa-multilateration/
* https://www.mdpi.com/1424-8220/20/7/1842

## Example solving navigation equations

In [ ]:
from scipy.optimize import fsolve
import math

def equations(p):
    x, y,z,d = p
    PosDict = {1: {'X': 0, 'Y': 0, 'Z': 2},
               2: {'X': 0, 'Y': 6, 'Z': 2},
               3: {'X': 8, 'Y': 6, 'Z': 2},
               4: {'X': 8, 'Y': 0, 'Z': 2}}

    A = PosDict[1]
    B = PosDict[2]
    C = PosDict[3]
    D = PosDict[4]
    t = [m2s(5),m2s(5),m2s(5),m2s(5)]
    return (((x-A['X'])**2 + (y-A['Y'])**2 + (z-A['Z'])**2 - (c*(t[0] - d))**2),
            ((x-B['X'])**2 + (y-B['Y'])**2 + (z-B['Z'])**2 - (c*(t[1] - d))**2),
            ((x-C['X'])**2 + (y-C['Y'])**2 + (z-C['Z'])**2 - (c*(t[2] - d))**2),
            ((x-D['X'])**2 + (y-D['Y'])**2 + (z-D['Z'])**2 - (c*(t[3] - d))**2))


results =  fsolve(equations, [0,0,0,0] )
x, y,z,d = results

print(f"x: {x}, y: {y} z: {z}, delay: {d*1e9:2.5}ns or :{s2m(d):.4}m")


np.isclose(equations(results), [0, 0, 0 ,0])
